# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 29

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,90s,Gls,Ast,G-PK,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK.1,G+A-PK,xG,npxG,xA,npxG+xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA.1,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,30-216,1990,18,17,1504,16.7,0,1,0,0,0,2,0,0.00,0.06,0.06,0.00,0.06,1.2,1.2,0.8,2.0,0.07,0.05,0.12,0.07,0.12,Matches
2,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,23-182,1997,20,12,1021,11.3,6,1,6,0,0,0,0,0.53,0.09,0.62,0.53,0.62,5.6,5.6,0.9,6.5,0.49,0.08,0.57,0.49,0.57,Matches
3,Che Adams\Che-Adams,sco SCO,FW,Southampton,24-263,1996,28,24,2165,24.1,7,4,7,0,0,1,0,0.29,0.17,0.46,0.29,0.46,6.5,6.5,4.6,11.1,0.27,0.19,0.46,0.27,0.46,Matches
4,Tosin Adarabioyo\Tosin-Adarabioyo,eng ENG,DF,Fulham,23-190,1997,26,26,2340,26.0,0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,1.0,1.0,0.3,1.3,0.04,0.01,0.05,0.04,0.05,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,34-089,1987,3,3,270,3.0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Manchester City,1,30,22,5,3,64,21,43,71,58.1,23.6,34.5,1.15,W W L W W,NaN,İlkay Gündoğan - 12,Ederson,NaN
Manchester Utd,2,29,16,9,4,56,32,24,57,45.5,34.1,11.4,0.39,W D D W W,NaN,Bruno Fernandes - 16,David de Gea,NaN
Leicester City,3,29,17,5,7,53,32,21,56,44.0,34.5,9.5,0.33,W L D W W,NaN,Jamie Vardy - 12,Kasper Schmeichel,NaN
Chelsea,4,29,14,9,6,44,25,19,51,46.0,24.5,21.5,0.74,D D W W D,143.0,Tammy Abraham Jorginho - 6,Edouard Mendy,NaN
West Ham,5,29,14,7,8,45,35,10,49,40.3,33.8,6.5,0.23,W L W L D,133.0,Tomáš Souček - 9,Łukasz Fabiański,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,686,278,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,686,278,False,0.0,686,278,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,None,u,1,3,0.0,686,278,0,3441,0,54680,65,0.0,0.0,Özil,0
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,641,233,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,652,232,False,0.0,649,232,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,u,1,3,0.0,633,230,0,10266,0,19111,5,0.0,0.0,Sokratis,0
3,0,0,223,100.0,100.0,3,41270,NaN,,0,0,-1,1,45.2,324,105,4.0,,0,2,1.7,1.7,1,David,1.7,20,1,41.0,272,90,False,244.6,217,87,1344,,2021-02-02T20:30:22.388696Z,54,0,0,NaN,0,,41270.jpg,2.1,1,0,Luiz Moreira Marinho,0.9,False,None,a,1,3,123.0,219,61,40,76380,1079,115389,4044,0.3,7.4,David Luiz,1
4,1,10,296,100.0,100.0,7,54694,NaN,,0,0,-5,5,266.2,103,70,NaN,,2,3,3.3,3.3,2,Pierre-Emerick,3.3,23,9,140.7,41,27,False,414.0,108,37,1931,,2021-01-23T16:00:22.555453Z,115,1,0,1.0,0,,54694.jpg,4.4,0,0,Aubameyang,10.4,False,None,a,1,3,729.0,27,15,106,1110379,7419,3481073,125334,0.3,9.2,Aubameyang,2
5,1,3,121,100.0,100.0,2,58822,NaN,,0,0,-4,4,112.3,223,54,5.0,,0,2,0.7,0.7,0,Cédric,0.7,8,0,26.5,332,117,False,103.4,332,130,699,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,3.1,0,0,Soares,0.3,False,None,a,1,3,49.0,323,113,28,28937,264,51874,953,0.2,6.1,Cédric,1


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester City,1,30,22,5,3,64,21,43,71,58.1,23.6,34.5,1.15,W W L W W,NaN,İlkay Gündoğan - 12,Ederson,NaN,1.936667,0.786667,0.455360
Manchester Utd,2,29,16,9,4,56,32,24,57,45.5,34.1,11.4,0.39,W D D W W,NaN,Bruno Fernandes - 16,David de Gea,NaN,1.568966,1.175862,0.308553
Leicester City,3,29,17,5,7,53,32,21,56,44.0,34.5,9.5,0.33,W L D W W,NaN,Jamie Vardy - 12,Kasper Schmeichel,NaN,1.517241,1.189655,0.304326
Chelsea,4,29,14,9,6,44,25,19,51,46.0,24.5,21.5,0.74,D D W W D,143.0,Tammy Abraham Jorginho - 6,Edouard Mendy,NaN,1.586207,0.844828,0.429631
West Ham,5,29,14,7,8,45,35,10,49,40.3,33.8,6.5,0.23,W L W L D,133.0,Tomáš Souček - 9,Łukasz Fabiański,NaN,1.389655,1.165517,0.311761
Tottenham,6,29,14,6,9,49,30,19,48,39.7,33.7,6.1,0.21,W W W L W,143.0,Harry Kane - 17,Hugo Lloris,NaN,1.368966,1.162069,0.312838
Liverpool,7,29,13,7,9,48,36,12,46,51.8,35.4,16.4,0.57,L W L L W,400.0,Mohamed Salah - 17,Alisson,NaN,1.786207,1.220690,0.295027
Everton,8,28,14,4,10,40,37,3,46,33.5,38.7,-5.2,-0.19,W W W L L,143.0,Dominic Calvert-Lewin - 14,Jordan Pickford,NaN,1.196429,1.382143,0.251040
Arsenal,9,29,12,6,11,40,32,8,42,41.0,35.0,6.0,0.21,L W D W D,143.0,Alexandre Lacazette - 11,Bernd Leno,NaN,1.413793,1.206897,0.299124


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.7311111111111112

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,adjusted points,adjusted points per game,assists,assists_week1,assists_week10,assists_week11,assists_week12,assists_week13,assists_week14,assists_week15,assists_week16,assists_week17,assists_week18,assists_week19,assists_week2,assists_week20,assists_week21,assists_week22,assists_week23,assists_week24,assists_week25,assists_week26,assists_week27,assists_week28,assists_week3,assists_week4,assists_week5,assists_week6,assists_week7,assists_week8,assists_week9,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheet_points,clean_sheets,cleansheet_week1,cleansheet_week10,cleansheet_week11,cleansheet_week12,cleansheet_week13,cleansheet_week14,cleansheet_week15,cleansheet_week16,cleansheet_week17,cleansheet_week18,cleansheet_week19,cleansheet_week2,cleansheet_week20,...,xG_week17,xG_week18,xG_week19,xG_week2,xG_week20,xG_week21,xG_week22,xG_week23,xG_week24,xG_week25,xG_week26,xG_week27,xG_week28,xG_week3,xG_week4,xG_week5,xG_week6,xG_week7,xG_week8,xG_week9,xPoints,xPoints week 1,xPoints week 10,xPoints week 11,xPoints week 12,xPoints week 13,xPoints week 14,xPoints week 15,xPoints week 16,xPoints week 17,xPoints week 18,xPoints week 19,xPoints week 2,xPoints week 20,xPoints week 21,xPoints week 22,xPoints week 23,xPoints week 24,xPoints week 25,xPoints week 26,xPoints week 27,xPoints week 28,xPoints week 3,xPoints week 4,xPoints week 5,xPoints week 6,xPoints week 7,xPoints week 8,xPoints week 9,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,49.055048,2.767208,0,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0,206,100.0,100.0,22.055048,3,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,0.0,0.4,0.0,0.0,0.1,0.0,NaN,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,NaN,NaN,NaN,3.986341,NaN,0.000000,NaN,NaN,1.000000,3.397316,NaN,NaN,NaN,2.471518,7.363273,1.00000,3.204777,3.586388,-3.000000,NaN,4.395247,1.000000,5.026123,2.807586,3.986341,1.199148,5.274923,3.586388,1.000000,NaN,NaN,NaN,1
4,105.826801,4.579044,1,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,288,100.0,100.0,7.188312,7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.8,0.1,1.4,0.1,NaN,NaN,0.1,0.1,1.9,0.0,0.1,0.8,NaN,0.0,0.0,0.0,0.3,0.9,0.0,0.3,NaN,3.818731,4.246597,3.606531,1.70657,7.132871,NaN,NaN,5.449329,6.67032,2.867879,11.740818,5.61799,NaN,NaN,1.500000,1.500000,15.348812,2.122456,1.500000,7.201897,NaN,2.049787,2.818731,3.746597,4.101194,7.540818,2.765299,3.282085,2
5,32.935801,3.646464,1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,121,100.0,100.0,11.237311,2,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,...,NaN,NaN,0.0,NaN,0.0,0.1,0.0,0.0,0.1,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.986341,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,10.658585,NaN,3.804777,2.586388,1.986388,3.090127,4.095247,NaN,4.426123,NaN,3.986341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GD,Pts,xG,xGA,xGD,xGD/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Manchester City,1,30,22,5,3,64,21,43,71,58.2,23.7,34.5,1.15,W W L W W,NaN,İlkay Gündoğan - 12,Ederson,NaN,1.940000,0.790000,0.453845
Manchester Utd,2,29,16,9,4,56,32,24,57,45.6,34.1,11.5,0.40,W D D W W,NaN,Bruno Fernandes - 16,David de Gea,NaN,1.572414,1.175862,0.308553
Leicester City,3,29,17,5,7,53,32,21,56,44.1,34.5,9.5,0.33,W L D W W,NaN,Jamie Vardy - 12,Kasper Schmeichel,NaN,1.520690,1.189655,0.304326
Chelsea,4,29,14,9,6,44,25,19,51,46.1,24.5,21.6,0.75,D D W W D,143.0,Tammy Abraham Jorginho - 6,Edouard Mendy,NaN,1.589655,0.844828,0.429631
West Ham,5,28,14,6,8,42,32,10,48,38.1,32.1,6.0,0.21,W W L W L,143.0,Tomáš Souček - 8,Łukasz Fabiański,NaN,1.360714,1.146429,0.317770


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff',"n'lundulu",'carlosvinicius']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == "n'lundulu":
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('lundulu')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'carlosvinicius':
                exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('carlos')
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,assists_week21,xPoints week 22,xG_week22,goals_week22,xGA_week22,cleansheet_week22,xA_week22,assists_week22,xPoints week 23,xG_week23,goals_week23,xGA_week23,cleansheet_week23,xA_week23,assists_week23,xPoints week 24,xG_week24,goals_week24,xGA_week24,cleansheet_week24,xA_week24,assists_week24,xPoints week 25,xG_week25,goals_week25,xGA_week25,cleansheet_week25,xA_week25,assists_week25,xPoints week 26,xG_week26,goals_week26,xGA_week26,cleansheet_week26,xA_week26,assists_week26,xPoints week 27,xG_week27,goals_week27,xGA_week27,cleansheet_week27,xA_week27,assists_week27,xPoints week 28,xG_week28,goals_week28,xGA_week28,cleansheet_week28,xA_week28,assists_week28
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0.0,0.0,0,37605,NaN,,0,0,-3,3,0.0,686,278,NaN,,0,3,0.0,0.0,0,Mesut,0.0,0,0,0.0,686,278,False,0.0,686,278,0,Not included in Arsenal's 25-man Premier Leagu...,2020-10-20T22:30:18.118477Z,67,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.5,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.0,0.0,0,39476,NaN,,0,0,-2,2,0.0,641,233,NaN,,0,2,0.0,0.0,0,Sokratis,0.0,0,0,0.0,652,232,False,0.0,649,232,0,Left the club by mutual consent on 20/1,2020-10-21T10:30:18.546407Z,48,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,223,100.0,100.0,3,41270,NaN,,0,0,-1,1,45.2,324,105,4.0,,0,2,1.7,1.7,1,David,1.7,20,1,41.0,272,90,False,244.6,217,87,1344,,2021-02-02T20:30:22.388696Z,54,0,0,NaN,0,,41270.jpg,2.1,1,0,Luiz Moreira Marinho,0.9,False,...,0.0,-3.000000,0.0,0.0,1.4,0.0,0.0,0.0,NaN,NaN,NaN,1.3,0.0,NaN,NaN,4.395247,0.0,0.0,0.6,0.0,0.4,0.0,1.000000,0.0,0.0,2.1,0.0,0.0,0.0,5.026123,0.1,1.0,0.5,0.0,0.0,0.0,2.807586,0.0,0.0,1.6,0.0,0.0,0.0,3.986341,0.0,0.0,0.7,0.0,0.0,0.0
4,1,10,296,100.0,100.0,7,54694,NaN,,0,0,-5,5,266.2,103,70,NaN,,2,3,3.3,3.3,2,Pierre-Emerick,3.3,23,9,140.7,41,27,False,414.0,108,37,1931,,2021-01-23T16:00:22.555453Z,115,1,0,1.0,0,,54694.jpg,4.4,0,0,Aubameyang,10.4,False,...,NaN,1.500000,0.1,0.0,1.4,0.0,0.0,0.0,1.500000,0.1,0.0,1.3,0.0,0.0,0.0,15.348812,1.9,3.0,0.6,0.0,0.1,0.0,2.122456,0.0,0.0,2.1,0.0,0.0,0.0,1.500000,0.1,0.0,0.5,0.0,0.0,0.0,7.201897,0.8,1.0,1.6,0.0,0.0,0.0,NaN,NaN,NaN,0.7,0.0,NaN,NaN
5,1,3,121,100.0,100.0,2,58822,NaN,,0,0,-4,4,112.3,223,54,5.0,,0,2,0.7,0.7,0,Cédric,0.7,8,0,26.5,332,117,False,103.4,332,130,699,,2020-09-23T09:00:14.881983Z,46,0,0,NaN,0,,58822.jpg,3.1,0,0,Soares,0.3,False,...,0.0,1.986388,0.0,0.0,1.4,0.0,0.0,0.0,3.090127,0.0,0.0,1.3,0.0,0.0,0.0,4.095247,0.1,0.0,0.6,0.0,0.1,0.0,NaN,NaN,NaN,2.1,0.0,NaN,NaN,4.426123,0.0,0.0,0.5,0.0,0.0,0.0,NaN,NaN,NaN,1.6,0.0,NaN,NaN,3.986341,0.0,0.0,0.7,0.0,0.0,0.0


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
302,Fernandes,29.027778,209,7.2,188.416604,6.490907
272,De Bruyne,23.148148,125,5.4,147.374077,6.366560
388,Kane,26.901408,191,7.1,171.066667,6.359023
224,Vardy,24.915254,147,5.9,145.920000,5.856653
254,Salah,28.064516,174,6.2,161.826446,5.766230
271,Gündogan,22.903226,142,6.2,125.789216,5.492205
390,Son,27.968750,179,6.4,153.509695,5.488615
277,Cancelo,23.018868,122,5.3,122.747499,5.332473
301,Lingard,7.000000,56,8.0,36.622330,5.231761


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total goals: ' + str(total_goals))
print('Total xG: ' + str(total_xG))
print('goals per xG: ' + str(total_goals/total_xG))

Total goals: 727
Total xG: 748.3
goals per xG: 0.9715354804222905
